In [1]:
import os, re, json, gzip, pickle, shutil

from pydub.utils import mediainfo

In [2]:
%config IPCompleter.greedy=True
%config Completer.use_jedi = False

In [3]:
#myspace data downloaded from https://archive.org/download/myspace_dragon_hoard_2010 
myspace_metadata_path = '../data/myspace_mp3s/updated-metadata.tsv'
myspace_mp3s_path = '../data/myspace_mp3s'
metadata_path = '%s/metadata.json.gz' % myspace_mp3s_path
genre_cts_path = '%s/genre_cts.pkl' % myspace_mp3s_path
genre_map_path = '%s/genre_map.pkl' % myspace_mp3s_path
mapped_genre_cts_path = genre_cts_path.replace('genre', 'mapped_genre')

### Examples of raw metadata records

In [8]:
#8	The Oisters	136566144	the oi!sters	www.myspace.com/theoistersoioi	0	myspace	675	
#http://cache09-music01.myspacecdn.com/72/std_1e55dd8bd6213ae741667092fe272ea8.mp3	SALT LAKE CITY, Utrecht	
#Punk	1619	1605	840	2008-09-27
with open(myspace_metadata_path, encoding='ISO-8859-1') as f:
    for i in range(5):
        print(f.readline())

1	Big Yellow Moon	78393366	bill nelson	www.myspace.com/nelsonicausa	0	myspace	25796	http://cache06-music02.myspacecdn.com/46/std_1f69563352d19cb0132334cd0d3adeaf.mp3	Wakefield, Northwest	Rock,Electronica,Alternative	43248	41762	28382	2009-03-21

2	Flaming Desire	78393366	bill nelson	www.myspace.com/nelsonicausa	0	myspace	25796	http://cache08-music02.myspacecdn.com/59/std_db98343d3aaeecdb267624ed12443bfd.mp3	Wakefield, Northwest	Rock,Electronica,Alternative	43248	41762	28382	2009-03-21

3	Do You Dream In Colour	78393366	bill nelson	www.myspace.com/nelsonicausa	0	myspace	25796	http://cache04-music01.myspacecdn.com/30/std_a5d7357c06cbf627c53dc3a1e9524aa9.mp3	Wakefield, Northwest	Rock,Electronica,Alternative	43248	41762	28382	2009-03-21

4	A Buddha For My Brother	78393366	bill nelson	www.myspace.com/nelsonicausa	0	myspace	25796	http://cache10-music02.myspacecdn.com/80/std_cbd6477f38e09e1bb1a03cd02d699a73.mp3	Wakefield, Northwest	Rock,Electronica,Alternative	43248	41762	28382	2009-03-21

5	

### Extract fields of interest from metadata, format, and store

In [4]:
def parse_myspace_data(myspace_metadata_path, out_metadata_path, genre_cts_path):
    data = {}
    genre_cts = {}
    zip_cts = {}
    
    with open(myspace_metadata_path, encoding='ISO-8859-1') as f:
        for line in f:
            rec = {}
            
            mp3_filepath = ''
            
            line = line.replace('\n', '')
            items = line.split('\t')
            
            rec['song_name'] = format_text(items[1])
            rec['artist_name'] = format_text(items[3])
            
            #http://cache09-music01.myspacecdn.com/72/std_1e55dd8bd6213ae741667092fe272ea8.mp3
            zip_file, mp3_name = items[8].split('/')[-2:]
            
            rec['mp3_zipname'] = zip_file
            if zip_file not in zip_cts:
                zip_cts[zip_file]=0
            zip_cts[zip_file]+=1
            
            rec['mp3_filename'] = mp3_name
            
            mp3_filepath = '%s/%s' % (zip_file, mp3_name)
            
            genres = [format_text(g) for g in items[10].split(',')]
            for g in genres:
                if not g in genre_cts:
                    genre_cts[g]=0
                genre_cts[g]+=1
            rec['genres'] = genres
            
            data[mp3_filepath] = rec
            
    print('writing %d records' % len(data))
    with gzip.open(out_metadata_path, 'wt', encoding='utf-8') as oz:
        json.dump(data, oz)

    print('writing %d genre counts' % len(genre_cts))
    with open(genre_cts_path, 'wb') as o:
        pickle.dump(genre_cts, o)
            
    return myspace_data, genre_cts, zip_cts
            
    
def format_text(text):
    return text.strip().lower().replace(' ', '_').replace('&amp;', '&')


def load_metadata(path):
    with gzip.open(path, 'rt', encoding='utf-8') as fz:
        metadata = json.load(fz)
    return metadata


def load_genre_cts(path):
    with open(path, 'rb') as f:
        genre_cts = pickle.load(f)
    return genre_cts

In [5]:
if not os.path.exists(metadata_path):
    data, genre_cts, zip_cts = parse_myspace_data(myspace_metadata_path, metadata_path, genre_cts_path)
else:
    print('loading metadata and genre cts')
    data = load_metadata(metadata_path)
    genre_cts = load_genre_cts(genre_cts_path)
    
#(488550, 127, 490273), zips: 1-64 have almost 5K each, 65-80 have 8-9K each, 81+ have 5K each
len(data), len(genre_cts)

loading metadata and genre cts


(488550, 126)

### Normalize genres - map highly specific genres to general genres, e.g. 'hardcore' -> 'metal'
#### Note that each song can have up to 3 genres assigned

In [20]:
#raw genre counts

for g in sorted(genre_cts, key=genre_cts.get, reverse=True):
    print('%s\t%d' % (g, genre_cts[g]))

rock	116064
pop	58329
	58023
hip_hop	55815
alternative	47577
indie	47185
metal	36043
rap	35438
experimental	34023
electronica	33994
punk	33108
acoustic	32511
other	29262
r&b	26140
jazz	20789
folk	20677
electro	20514
hardcore	19669
soul	18380
progressive	16699
house	15645
techno	14071
blues	13902
psychedelic	13886
ambient	13456
reggae	13042
country	13039
folk_rock	12938
funk	11796
death_metal	11003
latin	9949
powerpop	8755
industrial	8241
trance	7954
thrash	7788
christian	7758
club	7724
classical	7373
garage	7276
pop_punk	6936
classic_rock	6816
gothic	6752
roots_music	6245
black_metal	6109
americana	5809
new_wave	5613
drum_&_bass	5304
ska	5247
dub	4996
down-tempo	4825
comedy	4682
breakbeat	4163
nu-jazz	3990
grindcore	3727
lounge	3556
melodramatic_popular_song	3556
lyrical	3545
afro-beat	3334
fusion	3320
disco_house	3285
crunk	3268
minimalist	3190
southern_rock	3106
rockabilly	3051
jam_band	3001
emo	2828
healing_&_easylistening	2813
electroacoustic	2784
surf	2757
trip_hop	2754
freestyle	

In [6]:
def map_genres(genre_cts, min_examples=1000):
    genre_map = {}
    
    #map subtypes to types with substrings
    substrs = ['punk', 'jazz', 'reggae', 'disco', 'house', 'rap', 'classical', 
               'metal', 'folk', 'pop', 'rock']
    for g in genre_cts:
        for substr in substrs:
            if substr in g and g not in genre_map:
                print('%s -> %s' % (g, substr))
                genre_map[g] = substr
                
    genre_map['rockabilly'] = 'oldies'
    print('rockabilly -> oldies')
    
    #various other mappings
    map_to = 'metal'
    for g in ['hardcore', 'thrash', 'grindcore', 'screamo', 'happy_hardcore']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    map_to = 'punk'
    for g in ['psychobilly']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    map_to = 'classical'
    for g in ["a'cappella", 'choral', 'chamber_music', 'baroque', 'medieval', 'renaissance', 'symphonic', 
              'opera', 'vocal', 'ensemble', 'piano', 'guitar', 'solo_instruments', 'woodwinds', 'strings']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    map_to = 'hip_hop'
    for g in ['crunk', 'trip_hop', 'hyphy', 'west_coast', 'east_coast', 'contemporary_urban']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to      
            
    map_to = 'dance'
    for g in ['club', 'idm', '2-step', 'big_beat', 'grime', 'breakcore', 'freestyle', 'trance', 'techno',
              'turntablism', 'breakbeat', 'dub', 'drum_&_bass', 'ghettotech']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to  
    
    map_to = 'electronic'
    for g in ['electronica', 'electro', 'live_electronics', 'emotronic']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to        
    
    map_to = 'alternative'
    for g in ['grunge', 'ska', 'garage', 'shoegaze', 'indie', 'alternative_general']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
    
    map_to = 'oldies'
    for g in ['rockabilly', 'surf', 'soft_rock', 'adult_contemporary', 'swing', 'swing/big_band', 'western_swing',
              'love_songs', 'lounge', 'aaa/adult_alternative', 'oldies', 'standards', 
              'crooners/vocals', 'romantic', 'rock-n-roll_oldies']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    map_to = 'jazz'
    for g in ['soul', 'neo-soul', 'fusion', 'world_fusion', 'detroit', 'bebop', 'mod']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    map_to = 'blues'
    for g in ['r&b']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    map_to = 'rock'
    for g in ['psychedelic', 'acid', 'glam', 'jam_band']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to

    map_to = 'spiritual'
    for g in ['gospel', 'spiritual', 'christian', 'religious']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    map_to = 'pop'
    for g in ['idol', 'lyrical', 'contemporary', 'crossover', 'alternative_cover_songs']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    map_to = 'ambient'
    for g in ['down-tempo', 'mood_music', 'new_age']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
    
    map_to = 'experimental'
    for g in ['concrete', 'visual', 'minimal', 'minimalist', 'experimental', 'noise', 
              'abstract', 'tape_music', 'acousmatic', 'electroacoustic']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    map_to = 'funk'
    for g in ['funk', 'funky_breaks', 'groove']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    map_to = 'goth'
    for g in ['goth_rock', 'darkwave', 'new_wave', 'emo', 'goth', 'gothic']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    map_to = 'rap'
    for g in ['new_school', 'old_school']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
    
    #just lumping these together
    map_to = 'world'
    for g in ['zouk', 'chinese_traditional', 'japanese_classic_music', 'salsa', 'samba', 'latin', 'flamenco',
              'bossa_nova', 'jungle', 'regional_mexican', 'celtic', 'tropical', 'afro-beat', 'roots_music',
              'world/folk_cover_songs', 'general_latin', 'asian', 'americana', 'world_traditions', 
              'native_american', 'tribal', 'caribbean', 'african', 'brazilian',  'hawaiian',
              'russian', 'indian', 'jewish/israeli', 'french', 'nouveau_flamenco', 'cuban', 'arabic',
              'merengue', 'scandinavian', 'european', 'south/central_american', 'tango',
              'traditional', 'quebecois', ]:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
    #map types of spoken word
    map_to = 'spoken_word'
    for g in ['humor', 'comedy', 'parodies', 'general_comedy', 'political_humor', 'satire', 'spoofs', 'bodily_functions', 
              'dumb_stories', 'lies', 'speak_your_mind', 'paranormal', 'aliens', 'witchcraft', 
              'self-help', 'mental_health', 'stories_and_myths', 'horror_stories', 'movies', 'audio_books', 
              'interviews', 'poetry', 'spoken_word', 'politics', 'shout_outs', 'commercials', 'recorded_greetings', 
              'opinions', 'education', 'emergency!', 'nonfiction', 
              'hypnosis', 'advice', 'radio', 'time_capsule_recordings', 'love']:
        if g not in genre_map:
            print('%s -> %s' % (g, map_to))
            genre_map[g] = map_to
            
            
    #direct remappings
    genre_map['healing_&_easylistening'] = 'easy_listening'
            
        
    #add remaining if there are enough instances
    for g,c in genre_cts.items():
        if not g.strip():
            continue
        if g not in genre_map:
            if min_examples and c<min_examples:
                continue
            print('%s -> %s' % (g, g))
            genre_map[g] = g
            
    return genre_map


def get_unique_mapped_genres(genre_map):
    return list(set([g for g in genre_map.values()]))

    
def get_mapped_genre_cts(mapped_genre_cts_path, genre_map):
    mapped_genre_cts = {g:0 for g in genre_map.values()}
    for g, mg in genre_map.items():
        if g not in genre_cts:
            continue
        mapped_genre_cts[mg]+=genre_cts[g]


    with open(mapped_genre_cts_path.replace('.pkl', '.txt'), 'w') as o:
        for g in sorted(mapped_genre_cts, key=mapped_genre_cts.get, reverse=True):
            print('%s\t%s' % (g, mapped_genre_cts[g]))
            o.write('%s\t%s\n' % (g, mapped_genre_cts[g]))

    with open(mapped_genre_cts_path, 'wb') as o:
        pickle.dump(mapped_genre_cts, o)
        
    return mapped_genre_cts


def read_genre_map(genre_map_path):
    with open(genre_map_path, 'rb') as f:
        genre_map = pickle.load(f)
    return genre_map


def write_genre_map(genre_map, genre_map_path):
    with open(genre_map_path, 'wb') as o:
         pickle.dump(genre_map, o)
    with open(genre_map_path.replace('.pkl', '.txt'), 'w') as o:
        for g in sorted(genre_map, key=genre_map.get):
            o.write('%s\t%s\n' % (g, genre_map[g]))

In [7]:
if os.path.exists(genre_map_path):
    print('loading genre map')
    genre_map = read_genre_map(genre_map_path)
else:
    genre_map = map_genres(genre_cts, min_examples=0)
    write_genre_map(genre_map, genre_map_path)

for rg,g in genre_map.items():
    print('%s -> %s' % (rg,g))

loading genre map
rock -> rock
punk -> punk
jazz -> jazz
pop -> pop
rap -> rap
reggae -> reggae
black_metal -> metal
folk -> folk
metal -> metal
death_metal -> metal
j-pop -> pop
post_punk -> punk
house -> house
classical_-_opera_and_vocal -> classical
folk_rock -> folk
hard_house -> house
pop_punk -> punk
powerpop -> pop
classical -> classical
rockabilly -> oldies
nu-jazz -> jazz
classic_rock -> rock
italian_pop -> pop
spanish_pop -> pop
disco_house -> disco
melodramatic_popular_song -> pop
southern_rock -> rock
reggaeton -> reggae
chinese_pop -> pop
k-pop -> pop
german_pop -> pop
progrsv_house -> house
christian_rap -> rap
dutch_pop -> pop
french_pop -> pop
hardcore -> metal
thrash -> metal
grindcore -> metal
screamo -> metal
happy_hardcore -> metal
psychobilly -> punk
a'cappella -> classical
choral -> classical
chamber_music -> classical
baroque -> classical
medieval -> classical
renaissance -> classical
symphonic -> classical
opera -> classical
vocal -> classical
ensemble -> classi

In [8]:
#unique mapped genres

gs = get_unique_mapped_genres(genre_map)
len(gs), gs

(32,
 ['progressive',
  'punk',
  'showtunes',
  'folk',
  'house',
  'metal',
  'rap',
  'dance',
  'classical',
  'ambient',
  'country',
  'acoustic',
  'world',
  'blues',
  'hip_hop',
  'oldies',
  'funk',
  'jazz',
  'disco',
  'reggae',
  'electronic',
  'experimental',
  'industrial',
  'goth',
  'alternative',
  'spoken_word',
  'pop',
  'spiritual',
  'rock',
  'bluegrass',
  'other',
  'easy_listening'])

In [9]:
# get counts for mapped genres

if not os.path.exists(mapped_genre_cts_path):
    mapped_genre_cts = get_mapped_genre_cts(mapped_genre_cts_path, genre_map)
else:
    print('loading mapped genre cts')
    with open(mapped_genre_cts_path, 'rb') as f:
        mapped_genre_cts = pickle.load(f)
len(mapped_genre_cts)

loading mapped genre cts


32

### Unzip a test directory and see if metadata recs can be mapped to songs

In [65]:
def search_zip(folder_path, folder_name, data, genre_map):
    missing = []
    found = []
    genres = {}
    comb_genres={}
    for fn, meta in data.items():
        if fn.startswith('%s/' % folder_name):
            fp = '%s/%s' % (folder_path, fn)
            if not os.path.exists(fp):
                missing.append(meta)
            else:
                found.append(meta)
                if not 'genres' in meta:
                    continue
                
                for g in meta['genres']:
                    if not g:
                        continue
                    g = genre_map[g]
                    if not g in genres:
                        genres[g]=0
                    genres[g]+=1
                
                gs = ', '.join(set([genre_map[g] for g in meta['genres'] if g]))
                if not gs in comb_genres:
                    comb_genres[gs]=0
                comb_genres[gs]+=1
                
    print('missing: %d, found: %d, genres: %d, genre combos: %d' % (len(missing), len(found), len(genres), len(comb_genres)))
    
    return missing, found, genres, comb_genres

In [66]:
folder_name='1'
missing, found, genres, comb_genres = search_zip(myspace_mp3s_path, folder_name, data, genre_map)

missing: 101, found: 4705, genres: 32, genre combos: 1062


In [63]:
#genre counts for this zip
[(g, genres[g]) for g in sorted(genres, key=genres.get, reverse=True)]

[('rock', 1378),
 ('alternative', 1049),
 ('metal', 944),
 ('pop', 701),
 ('electronic', 534),
 ('dance', 515),
 ('hip_hop', 499),
 ('jazz', 460),
 ('punk', 441),
 ('experimental', 409),
 ('world', 364),
 ('blues', 332),
 ('folk', 319),
 ('acoustic', 302),
 ('rap', 285),
 ('other', 280),
 ('progressive', 163),
 ('ambient', 163),
 ('goth', 158),
 ('house', 151),
 ('reggae', 133),
 ('funk', 116),
 ('country', 115),
 ('classical', 99),
 ('spiritual', 92),
 ('oldies', 90),
 ('industrial', 85),
 ('spoken_word', 47),
 ('disco', 30),
 ('bluegrass', 25),
 ('easy_listening', 18),
 ('showtunes', 10)]

In [67]:
#unique assigned genre combination counts
[(g, comb_genres[g]) for g in sorted(comb_genres, key=comb_genres.get, reverse=True)][:100]

[('', 699),
 ('metal', 204),
 ('rock', 116),
 ('alternative, rock', 80),
 ('hip_hop, rap', 65),
 ('other', 53),
 ('punk, metal', 53),
 ('alternative, pop, rock', 49),
 ('alternative', 49),
 ('hip_hop, blues, rap', 48),
 ('rock, metal', 47),
 ('hip_hop', 45),
 ('rock, alternative', 43),
 ('pop', 43),
 ('electronic, dance', 42),
 ('punk', 41),
 ('electronic, house, dance', 36),
 ('rock, pop', 35),
 ('jazz', 34),
 ('rock, pop, alternative', 34),
 ('dance', 33),
 ('world', 30),
 ('reggae', 28),
 ('rock, punk', 26),
 ('rock, punk, metal', 25),
 ('rock, punk, alternative', 24),
 ('progressive, metal', 24),
 ('rap', 23),
 ('alternative, acoustic, folk', 23),
 ('rock, alternative, metal', 23),
 ('electronic', 22),
 ('alternative, punk', 21),
 ('alternative, pop', 19),
 ('acoustic', 19),
 ('blues', 18),
 ('experimental', 18),
 ('hip_hop, rap, dance', 18),
 ('rock, progressive', 17),
 ('rock, pop, acoustic', 17),
 ('house, dance', 16),
 ('folk', 16),
 ('blues, jazz', 16),
 ('alternative, rock, p

### Sort mp3s into genre folders by majority mapped genre

In [10]:
def sort_into_genre_folders(folder_path, folder_name, data, genre_map, mapped_genre_cts):
    genres = {g:0 for g in genre_map.values()}
    for fn, meta in data.items():
        zip_number, fn = fn.split('/')
        if zip_number == folder_name:
            fp = '%s/%s/%s' % (folder_path, zip_number, fn)
            if not os.path.exists(fp):
                continue
            if not 'genres' in meta or not meta['genres']:
                continue

            g = [genre_map[g] for g in meta['genres'] if g in genre_map]
            if not g:
                continue
                
            use_genre = g[0]
            for gn in g:
                if g.count(gn)>1:
                    use_genre = gn
                    break
            g = use_genre
            
            genres[g]+=1
            gp = '%s/%s/%s' % (folder_path, zip_number, g)
            if not os.path.exists(gp):
                print('make %s for %s with %s' % (gp, fn, meta['genres']))
                os.mkdir(gp)
            nfp = '%s/%s' % (gp, fn)
            shutil.move(fp, nfp)
            
    os.mkdir('%s/%s/unknown' % (folder_path, folder_name))
    for fn in os.listdir('%s/%s' % (folder_path, folder_name)):
        if fn.endswith('.mp3'):
            fp = '%s/%s/%s' % (folder_path, folder_name, fn)
            nfp = '%s/%s/unknown/%s' % (folder_path, folder_name, fn)
            shutil.move(fp, nfp)
            
    print([(g, genres[g]) for g in sorted(genres, key=genres.get, reverse=True)])
    
    
def resort_into_genre_folders(folder_path, folder_name, metadata, genre_map):
    genres = {g:0 for g in genre_map.values()}
    moved=0
    same=0
    zip_path = folder_path + '/' + folder_name
    for genre in os.listdir(zip_path):
        if genre=='unknown':
            continue
        
        genre_path = '%s/%s' % (zip_path, genre)
        print(genre_path)
        
        for fn in os.listdir(genre_path):
            if not fn.endswith('.mp3'):
                continue

            fp = '%s/%s' % (genre_path, fn)

            zip_num_filename = '%s/%s' % (folder_name, fn)
            if zip_num_filename not in metadata:
                print('not in meta', zip_num_filename)
                continue

            meta = metadata[zip_num_filename]
            if not 'genres' in meta or not meta['genres']:
                continue

            g = [genre_map[g] for g in meta['genres'] if g in genre_map]
            if not g:
                continue

            use_genre = g[0]
            for gn in g:
                if g.count(gn)>1:
                    use_genre = gn
                    break
            g = use_genre
            genres[g]+=1
            
            gp = '%s/%s' % (zip_path, g)
            
            if not os.path.exists(gp):
                print('make %s for %s with %s' % (gp, fn, meta['genres']))
                os.mkdir(gp)
                
            nfp = '%s/%s' % (gp, fn)
            if not fp==nfp:
                moved+=1
                print('moving', fp,nfp)
                shutil.move(fp, nfp)
            else:
                same+=1
            
    print('same',same,'moved', moved, [(g, genres[g]) for g in sorted(genres, key=genres.get, reverse=True)])

In [12]:
folder_name = '78'
sort_into_genre_folders(myspace_mp3s_path, folder_name, data, genre_map, mapped_genre_cts)
#resort_into_genre_folders(myspace_mp3s_path, folder_name, data, genre_map)

make ../data/myspace_mp3s/78/oldies for std_4bf3ebe144cfab872e7cb43b9ec9fc55.mp3 with ['lounge']
make ../data/myspace_mp3s/78/dance for std_e572d0535f38b8308859d947046db8d6.mp3 with ['club', 'techno', 'electronica']
make ../data/myspace_mp3s/78/alternative for std_01ef82f028003febd357313487187cb5.mp3 with ['indie', 'rock', 'alternative']
make ../data/myspace_mp3s/78/country for std_4aec55c1ef9cf8d0fb912af95446f34d.mp3 with ['country']
make ../data/myspace_mp3s/78/rap for std_aec2502e7bae03238d7ce65b39f51cbf.mp3 with ['rap']
make ../data/myspace_mp3s/78/punk for std_9b3cfa92d088e951cdc28a7c3a0769df.mp3 with ['pop_punk']
make ../data/myspace_mp3s/78/hip_hop for std_6ff68840058176f92bf875c023aad775.mp3 with ['hip_hop', 'drum_&_bass', 'funk']
make ../data/myspace_mp3s/78/jazz for std_25b5aadf629fddf11495c45b92d407da.mp3 with ['soul', 'hip_hop', 'electronica']
make ../data/myspace_mp3s/78/metal for std_9f548292754db18eb8662937b4280561.mp3 with ['metal', 'hardcore', 'rock']
make ../data/mysp

In [97]:
for fn in os.listdir('%s/%s/metal' % (myspace_mp3s_path, folder_name)):
    fp = '%s/%s' % (folder_name, fn)
    if fp not in data:
        print(fp, 'not found in data')
        continue
    meta = data[fp]
    print('%s\t%s\t%s\t%s' % (fn, meta['artist_name'], meta['song_name'], meta['genres']))

std_00302ad2489630819f64a2c35aa9729d.mp3	hell_within	shadows_of_vanity	['metal', 'hardcore', 'thrash']
std_0058505adce0279aaf5a6f528aebc516.mp3	eden_weint_im_grab	undine_sey_meyn_schutzgeyst	['metal', 'black_metal', 'gothic']
std_006c145cea29e3c93ab331c882d82f97.mp3	slobodan_kajkut	20_minutes_for_2000_euros	['thrash']
std_008eb15c3483e41094c623e3e4978a37.mp3	crisis_bureau	automated_response	['hardcore', 'punk']
std_01314a7de97020d6e9ed1f03f8023ebd.mp3	temple_of_baal	living_fleshthrone	['death_metal', 'black_metal', 'thrash']
std_017d2af5d22af87b62865d9fd8bc65ea.mp3	before_the_dawn	my_darkness	['metal', 'progressive', 'gothic']
std_01be1a490d6a169801e864080b962244.mp3	acrid_asphyxiation_(rip)	intestine_jigsaw_puzzle	['grindcore', 'hardcore', 'death_metal']
std_01c73027bb5c37ba1230b909a5ff4766.mp3	odd_squad	welcome_wagon	['hardcore', 'punk']
std_01d41805d0565ad89b2bf0a2069e681d.mp3	membranes_of_the_multiverse	frustration	['grindcore', 'hardcore', 'death_metal']
std_024f49f17df0758d84be4e

In [25]:
for meta in found:
    if 'black_metal' in meta['genres'] or 'death_metal' in meta['genres']:
        print('\n\n', meta)



 {'song_name': 'leviticus', 'artist_name': 'rwake', 'mp3_zipname': '1', 'mp3_filename': 'std_c0878c4c03e671ad4e219c6cb7321e67.mp3', 'genres': ['black_metal', 'progressive', 'southern_rock']}


 {'song_name': 'into_death', 'artist_name': 'curriculum_mortis', 'mp3_zipname': '1', 'mp3_filename': 'std_8821d641f8970501f3873cfdfabc912d.mp3', 'genres': ['death_metal']}


 {'song_name': 'ele', 'artist_name': 'mosquitos', 'mp3_zipname': '1', 'mp3_filename': 'std_2c9f1a491d4c570428c631df2a446416.mp3', 'genres': ['grime', 'ghettotech', 'death_metal']}


 {'song_name': 'benedicto_xvi', 'artist_name': 'sophya', 'mp3_zipname': '1', 'mp3_filename': 'std_3be8bca056166e0b57bb1f75e3655685.mp3', 'genres': ['metal', 'death_metal', 'black_metal']}


 {'song_name': 'cankerous', 'artist_name': 'bane_of_existence', 'mp3_zipname': '1', 'mp3_filename': 'std_d35af2c0635c37bb26be419ae5a7416e.mp3', 'genres': ['death_metal']}


 {'song_name': 'sodomized_by_god', 'artist_name': 'willow_wisp?(rip_traci-a_true_drum_

In [ ]:
#_-_new_song_up! _(searching_dates_!!!)  _new_blog_up!  _(live_pics_online!)  _(auditioning_bassists)
# _(album_out_now!!)  _{buy_our_cd_today!_}   _pics_up!!!!  _(suche_drummer!!!)   _(two_new_songs_up)
#_(has_a_song_up!)  _-_looking_for_second_cd_reedition!  __-___(the_atrocity_exhibition__out_now!_)
#_[writing_new_songs]  _-_searching_drummer_&_new_song_online!!!  _(debut_album_out_now!!)
#_(in_writing_mode)  _(r.i.p.) _-_searching_for_a_label  _(final_show_tba!!!)  _-_new_songs!  _(new_songs!)
#_(in_search_of_label...)  _recording_new_album  _(writing_new_music)  _(is_coming_back?)  _(in_studio)

In [43]:
#fp = '%s/1/std_b0317cf9e69e6baaf0ec274b75951bcc.mp3' % myspace_mp3s_path
fp = '%s/1/std_cd01d0fe6361f924458668c06e975a0f.mp3' % myspace_mp3s_path
metadata = mediainfo(fp).get('TAG', None)
metadata

{'encoder': 'LAME3.96r'}

In [ ]:
def add_size_to_data(data, mp3s_path, metadata_path):
    ttl = 0
    not_found = []
    over_5mb = 0
    
    for i, fn in enumerate(data.keys()):
        if i and i%10000==0:
            print('Records checked so far: %d\tRecs found: %d\t> 5 Mb: %d\tNot Found: %d, including %s' % (
                i, ttl, over_5mb, len(not_found), not_found[-1]))
            with gzip.open(metadata_path, 'wt', encoding='utf-8') as oz:
                json.dump(data, oz)
                
        fp = '%s/%s' % (mp3s_path, fn)
        if not os.path.exists(fp):
            not_found.append(fn)
            continue
            
        ttl+=1
            
        data[fn]['size_mb'] = os.path.getsize(fp)/1024**2
        if data[fn]['size_mb'] > 5:
            over_5mb+=1
            
    return data

In [ ]:
data = add_size_to_data(data, mp3s_path, metadata_path)